<a href="https://colab.research.google.com/github/Air000/my-ReID/blob/main/CLIP_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import CLIPProcessor, CLIPModel

# Load pre-trained CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip drive/MyDrive/tmp/PhD_project/CLIP-ReID/data/T2I-VeRi.zip

Archive:  drive/MyDrive/tmp/PhD_project/CLIP-ReID/data/T2I-VeRi.zip
   creating: T2I-VeRi/
   creating: T2I-VeRi/image/
  inflating: T2I-VeRi/image/0001_c013_00014800_0.jpg  
  inflating: T2I-VeRi/image/0001_c014_00013175_0.jpg  
  inflating: T2I-VeRi/image/0001_c016_00012920_1.jpg  
  inflating: T2I-VeRi/image/0001_c019_00013820_0.jpg  
  inflating: T2I-VeRi/image/0002_c002_00030615_1.jpg  
  inflating: T2I-VeRi/image/0002_c004_00084270_0.jpg  
  inflating: T2I-VeRi/image/0002_c005_00084980_0.jpg  
  inflating: T2I-VeRi/image/0002_c008_00084505_0.jpg  
  inflating: T2I-VeRi/image/0003_c001_00021535_0.jpg  
  inflating: T2I-VeRi/image/0003_c015_00018755_0.jpg  
  inflating: T2I-VeRi/image/0003_c016_00018465_0.jpg  
  inflating: T2I-VeRi/image/0004_c001_00043990_0.jpg  
  inflating: T2I-VeRi/image/0004_c013_00044295_0.jpg  
  inflating: T2I-VeRi/image/0005_c004_00077650_0.jpg  
  inflating: T2I-VeRi/image/0005_c005_00078360_0.jpg  
  inflating: T2I-VeRi/image/0005_c007_00078460_0.jpg  


In [5]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import os.path as osp
import glob
import json
import re

class T2IDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, cid, vid, cap = self.dataset[index]
        img = Image.open(img_path).convert("RGB")

        if self.processor is not None:
            inputs = self.processor(text=cap, images=img, return_tensors="pt", padding="max_length", truncation=True, max_length=77)
            # Print the shapes of individual tensors within 'inputs'
            # print("Pixel Values Shape:", inputs['pixel_values'].shape)
            # print("Input IDs Shape:", inputs['input_ids'].shape)
            # print("Attention Mask Shape:", inputs['attention_mask'].shape)
        # cap = clip.tokenize(cap)
        return inputs

class CustomCLIPDataset(object):
    dataset_dir = 'T2I-VeRi'

    def __init__(self, root='', verbose=True, **kwargs):
        super(CustomCLIPDataset, self).__init__()
        self.dataset_dir = osp.join(root, self.dataset_dir)
        self.image_dir = osp.join(self.dataset_dir, 'image')
        self.split_path = osp.join(self.dataset_dir, 'reid.json')

        self._check_before_run()

        # Process the splits
        train, query, gallery = self._process_split()

        self.train = train
        self.query = query
        self.gallery = gallery

    def _check_before_run(self):
        """Check if all files are available before going deeper"""
        if not osp.exists(self.dataset_dir):
            raise RuntimeError("'{}' is not available".format(self.dataset_dir))
        if not osp.exists(self.image_dir):
            raise RuntimeError("'{}' is not available".format(self.image_dir))
        if not osp.exists(self.split_path):
            raise RuntimeError("'{}' is not available".format(self.split_path))

    def _process_split(self):
        with open(self.split_path, 'r') as f:
            data = json.load(f)

        train_set = []
        test_set = []

        for item in data:
            img_path = osp.join(self.image_dir, osp.basename(item['file_path']))
            pid = item['id']
            split = item['split']
            captions = item['captions'][0]
            camid = int(re.search(r'_c(\d+)', item['file_path']).group(1))
            camid -= 1  # index starts from 0

            if split == 'train':
                train_set.append((img_path, pid, camid, 0, captions))  # viewid is set to 0 for simplicity
            elif split == 'test':
                test_set.append((img_path, pid, camid, 0, captions))

            # Split test set into query and gallery
            query_set = []
            gallery_set = []
            pid_to_images = {}

            # Group images by pid
            for img_path, pid, camid, viewid, captions in test_set:
                if pid not in pid_to_images:
                    pid_to_images[pid] = []
                pid_to_images[pid].append((img_path, camid, viewid, captions))

            # For each pid, randomly assign one image to the query set and the rest to the gallery set
            for pid, images in pid_to_images.items():
                query_set.append((images[0][0], pid, images[0][1], images[0][2], images[0][3]))
                for img in images[1:]:
                    gallery_set.append((img[0], pid, img[1], img[2], img[3]))

        return train_set, query_set, gallery_set

# Example dataset

dataset = CustomCLIPDataset(root="./")
dataset_train = T2IDataset(dataset.train, processor)
dataloader = DataLoader(dataset_train, batch_size=16, shuffle=True)

In [6]:
import torch.nn.functional as F

def clip_loss(image_features, text_features, temperature=0.07):
    """
    Compute CLIP contrastive loss for image and text features.

    :param image_features: Image embeddings (batch_size, embedding_dim)
    :param text_features: Text embeddings (batch_size, embedding_dim)
    :param temperature: Scaling factor
    :return: Contrastive loss
    """
    # Normalize embeddings
    image_features = F.normalize(image_features, p=2, dim=-1)
    text_features = F.normalize(text_features, p=2, dim=-1)

    # Compute similarity matrix (batch_size x batch_size)
    logits = (image_features @ text_features.T) / temperature

    # Ground truth labels (each image matches only its corresponding text)
    batch_size = image_features.shape[0]
    labels = torch.arange(batch_size, dtype=torch.long, device=image_features.device)

    # Compute loss
    loss = (F.cross_entropy(logits, labels) + F.cross_entropy(logits.T, labels)) / 2
    return loss

In [7]:
import torch.optim as optim

# Define optimizer
optimizer = optim.AdamW(model.parameters(), lr=5e-6, weight_decay=0.01)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        # Move data to GPU if available
        inputs = {key: value.to(device) for key, value in batch.items()}

        # Get features from CLIP
        # Access pixel_values and input_ids tensors directly
        text_features = model.get_text_features(inputs['input_ids'].squeeze(1))  # squeeze to remove extra dim
        image_features = model.get_image_features(inputs['pixel_values'].squeeze(1)) # squeeze to remove extra dim

        # Compute contrastive loss
        loss = clip_loss(image_features, text_features)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader)}")

# Save fine-tuned model
model.save_pretrained("fine_tuned_clip_model")
processor.save_pretrained("fine_tuned_clip_processor")

Epoch 1/5, Loss: 1.0598166368970083
Epoch 2/5, Loss: 0.5305222274240003
Epoch 3/5, Loss: 0.32052811419745103
Epoch 4/5, Loss: 0.21890079462473547
Epoch 5/5, Loss: 0.14421414067849106


[]